# Example: Eigendecomposition of Stoichiometric Matrices
In this example, we'll explore the eigendecomposition of a standard matrix in chemical systems: the stoichiometric matrix $\mathbf{S}$. The stoichiometric matrix is the digital representation of a chemical reaction system.

### What is a stoichiometric matrix?

Suppose we have a set of chemical (or biochemical) reactions $\mathcal{R}$ involving the chemical species (metabolite) set $\mathcal{M}$. Then, the stoichiometric matrix is a $\mathbf{S}\in\mathbb{R}^{|\mathcal{M}|\times|\mathcal{R}|}$ matrix that holds the stoichiometric coefficients $\sigma_{ij}\in\mathbf{S}$ such that:
* $\sigma_{ij}>0$: Chemical species (metabolite) $i$ is _produced_ by reaction $j$. Species $i$ is a product of reaction $j$.
* $\sigma_{ij} = 0$: Chemical species (metabolite) $i$ is not connected with reaction $j$
* $\sigma_{ij}<0$: Chemical species (metabolite) $i$ is _consumed_ by reaction $j$. Species $i$ is a reactant of reaction $j$.


### Tasks

Before we start, divide into teams and familiarize yourself with the lab. Then, execute the `Run All Cells` command to check if you (or your neighbor) have any code or setup issues. Code issues, then raise your hands - and let's get those fixed!

* __Task 1: Setup, Data, Constants (20 min)__: Let's take 20 minutes to review the dataset we'll explore today and set up some values we'll use in the other tasks. We'll load the data and do some initial _data munging_ (also called [data wrangling](https://en.wikipedia.org/wiki/Data_wrangling)) to get the dataset in a form that we'll use in our analysis.
* __Task 2: Compute the eigendecomposition (10 min)__: In this task, we'll compute the eigendecomposition of the $\mathbf{\Sigma}$ matrix [using the `eigen(...)` method exported by the `LinearAlgebra.jl` package](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen) and some checks on the results.
* __Task 3: What does the largest eigenvalue/eigenvector tell us (20 min)?__: Now that we have the eigenvalues and eigenvectors, in this task, we'll explore what the largest eigenvector coefficients tell us about the $\mathbf{\Sigma}$ matrix and, by extension, the stoichiometric matrix $\mathbf{S}$. Is there something interesting here?

Let's get started!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

In [ ]:
include(joinpath(@__DIR__, "Include.jl")); # include the Include.jl file

In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl). Check out [the documentation](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/) for more information on the functions, types, and data used in this material.

### Data
We developed a simple software development kit (SDK) against [the BiGG Models application programming interface at the University of California, San Diego](http://bigg.ucsd.edu/). The [BiGG Models database](http://bigg.ucsd.edu/) integrates published genome-scale metabolic networks into a single database with standardized nomenclature and structure. 

> __What are we doing here?__
> 
> We are going to download a stoichiometric matrix from [the BiGG models database](http://bigg.ucsd.edu/) using [the BiGG models API](http://bigg.ucsd.edu/data_access) and then compute its eigendecomposition. 
> * [The BiGG models API](http://bigg.ucsd.edu/data_access) allows users to programmatically access genome-scale stoichiometric model reconstructions using a simple web API. There are `108` models of intracellular biochemistry occurring in various organisms (including humans) in the database (so far); [see here for a list of models](http://bigg.ucsd.edu/models).
> * Here, we'll first explore the [core metabolic model of Palsson and coworkers](https://pubmed.ncbi.nlm.nih.gov/26443778/), which is a scaled-down model of [carbohydrate metabolism](https://en.wikipedia.org/wiki/Carbohydrate_metabolism) in _E.coli_. This model has 72 metabolites and 95 reactions. We'll then look at other models, and see what is going on with these.
> 
>  We call the model download endpoint of [the BiGG models API](http://bigg.ucsd.edu/data_access) and then save the model file to disk (so we don't hit the API unless we have to). 

This call returns model information organized as [a Julia dictionary](https://docs.julialang.org/en/v1/base/collections/#Base.Dict) in the `model::Dict{String, Any}` variable. If a model file is saved, we use the cached file instead of making an API call.

In [ ]:
model = let

    # build download endpoint -
    baseurl = "http://bigg.ucsd.edu"; # base url to download model
    modelid = "iAB_RBC_283"; # model id to download
    path_to_saved_model_file = joinpath(_PATH_TO_DATA, "saved-model-$(modelid).jld2");

    # check: do we have a model file saved?
    model = nothing;
    if (isfile(path_to_saved_model_file) == false)
        
        endpoint = MyBiggModelsDownloadModelEndpointModel();
        endpoint.bigg_id = modelid;
        url = build(baseurl, endpoint)
        model = MyBiggModelsDownloadModelEndpointModel(url);

        # Before we move on, save this model for later (so we don't keep hitting the API)
        save(path_to_saved_model_file, Dict("model" => model));
    else
        model = load(path_to_saved_model_file)["model"];
    end
    model; # return the model (either saved, or downloaded)
end

__Metabolite records__: Each metabolite (chemical compound) in the network has an associated metabolite record with several fields. Let's take a look at the metabolite at index `1`. The key field for today in the metabolite record is the `id` field, an abbreviation or symbol associated with this metabolite.

In [ ]:
model["metabolites"][1] # example metabolite record

__Reaction records__: Similarly, each reaction in the network has a reaction record with several fields. Let's look at the reaction record at index `25`. The key field for the reaction record is the `metabolites` field, which lists the stoichiometric coefficients associated with this particular reaction.

In [ ]:
model["reactions"][25] # example reaction record

In [ ]:
model["reactions"][25]["metabolites"]

__Stoichiometric matrix__: Next, let's build a stoichiometric matrix $\mathbf{S}$ using the metabolite and reaction records. We'll do this using two for loops. 

> __Strategy__: In the outer loop, we iterate over the system's metabolites (chemical species) and select the `id` from the metabolites record for each metabolite. In the inner loop, we iterate over each reaction. For each reaction record, we ask if this reaction has an entry for the current metabolite `id` value; if it does, we grab the stoichiometric coefficient $\sigma_{ij}$ corresponding to this metabolite and reaction.

We'll save the stoichiometric matrix in the `S::Matrix{Float64}` variable.

In [ ]:
S = let

    # get some data from the model -
    m = model["metabolites"]; # get list of metabolites
    r = model["reactions"]; # get list of reactions
    number_of_rows = length(m); # how many metabolites do we have? (rows)
    number_of_cols = length(r); # how many reactions do we have? (cols)
    S = zeros(number_of_rows,number_of_cols); # initialize an empty stoichiometric matrix

    # let's build a stm -
    for i ∈ eachindex(m)
        metabolite = m[i]["id"]; # we are checking if this metabolite is in the reaction record
        for j ∈ eachindex(r)
            reaction = r[j];
            if (haskey(reaction["metabolites"], metabolite) == true)
                S[i,j] = reaction["metabolites"][metabolite];
            end
        end
    end
    S; 
end;

In [ ]:
S

___

## Task 1: Compute the covariance matrix
In this task, we'll compute the covariance matrix $\mathbf{\Sigma}$ of the reactions (columns) of the stoichiometric matrix. The stoichiometric matrix is not square. Thus, we cannot directly compute its eigendecomposition. However, suppose we compute [the covariance matrix](https://en.wikipedia.org/wiki/Covariance_matrix) between the columns, i.e., between the reactions stoichiometric vectors. This gives us an idea about the relationship between the network's reactions $i$ and $j$.

In [ ]:
Σ = cov(S) # this is col covariance matrix

__Idea__: Hmmm. The entries of the covariance matrix $\Sigma_{ij} = \sigma_{i}\sigma_{j}\rho_{ij}$ seem a little strange for stoichiometric vectors. Alternatively, suppose we defined another similarity matrix $\hat{\mathbf{\Sigma}}$ using [a Kernel function](https://en.wikipedia.org/wiki/Kernel_method) to compute each entry. 

> __Kernel function__:
> 
> A kernel function is a function that quantifies the similarity between two inputs (much more on this later). Here, we can use a kernel function to compute the similarity between two stoichiometric vectors (columns of the stoichiometric matrix $\mathbf{S}$). The $\hat{\mathbf{\Sigma}}$ matrix ahs entries $\hat{\Sigma}_{ij} = k(\mathbf{\sigma}_{i},\mathbf{\sigma}_{j})$, where $k:\mathbb{R}^{|\mathcal{M}|}\times\mathbb{R}^{|\mathcal{M}|}\rightarrow\mathbb{R}$ is [a Kernel function](https://en.wikipedia.org/wiki/Kernel_method) that takes two columns $\mathbf{\sigma}_{i}$ and $\mathbf{\sigma}_{j}$ for the stoichiometric matrix $\mathbf{S}$ and returns a _similarity score_. 

Let's try this out this idea using the functions exported by [the `KernelFunctions.jl` package](https://github.com/JuliaGaussianProcesses/KernelFunctions.jl) and see what happens.

In [ ]:
Σ̂ = let

    # initialize
    (number_of_metabolites, number_of_reactions) = size(S); # dimension of the system
    Σ̂ = zeros(number_of_reactions,number_of_reactions);

    d = SqExponentialKernel(); # what kernel function are we using?
    for i ∈ 1:number_of_reactions
        σᵢ = S[:,i]; # ith reaction
        for j ∈ 1:number_of_reactions
            σⱼ = S[:,j];
            Σ̂[i,j] = d(σᵢ,σⱼ)
        end
    end
    Σ̂
end

___

## Task 2: Compute the largest eigenvalue/eigenvector using power iteration
In this task, we'll compute the largest eigenvalue and associated eigenvector of the $\mathbf{\Sigma}$ matrix using the [power iteration method](https://en.wikipedia.org/wiki/Power_iteration). 

However, before we do that, let's use Julia's built-in [eigendecomposition function `eigen(...)`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen) to compute all the eigenvalues and eigenvectors of the $\mathbf{\Sigma}$ matrix (so we have something to compare against).

> __Why compute all the values__? We'll use [the `eigen(...)` method](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen) and then grab the last value(s) in task 2 where we look at the largest value, but we'll use the rest later. We'll also check the _magical properties_ of the eigendecomposition of symmetric real matrices; we need all the values for this check.

We'll store the eigenvalues in the diagonal $\mathbf{\Lambda}$ matrix, while the eigenvectors will be stored in the $\mathbf{V}$ matrix.

In [ ]:
Λ,V = let

    # initialize -
    A = Σ; # this is the matrix that we will decompose (if we are looking Σ̂, then change)
    (n,m) = size(A); # what is the dimension of A?
    Λ = Matrix{Float64}(1.0*I, n, n); # builds the I matrix, we'll update with λ -
    
    # Decompose using the built-in function
    F = eigen(A);   # eigenvalues and vectors in F of type Eigen
    λ = F.values;   # vector of eigenvalues
    V = F.vectors;  # n x n matrix of eigenvectors, each col is an eigenvector

    # package the eigenvalues into Λ -
    for i ∈ 1:n
        Λ[i,i] = λ[i];
    end

    Λ,V
end;

Fill me in

In [ ]:
λ₁,v₁ = let

    # initialize -
    A = Σ; # this is the matrix that we will decompose (free to change)
    (n,m) = size(A); # what is the dimension of A?
    v = ones(n); # initial guess for the eigenvector
    maxiter = 1000; # maxinum number of iterations
    ϵ = 0.00001; # tolerance

    # call poweriteration method
    result = poweriteration(A,v, maxiter = maxiter, ϵ = ϵ);

    # pull data from result 
    (result.value, result.vector)
end